Exercise 2 - Create a News Summary Tool
Objective: Build a tool that can fetch and summarize recent news articles.

Create a news summarization tool that works with the existing search functionality. This tool should take search results and create concise summaries of news articles.

Instructions:


Create a news_summarizer_tool that takes news content and creates summaries.
The tool should extract key information: headline, date, main points.
Format the output in a readable way.
Test it by asking the agent to "search for recent AI news and summarize the top 3 articles".
Starter Code:¶


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage, ToolMessage

from langgraph.graph import StateGraph, END

from typing import (Annotated,Sequence,TypedDict)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools import tool
import os
import json

In [ ]:

@tool
def news_summarizer_tool(news_content: str) -> str:
    """
    Summarize news articles from search results.
    
    :param news_content: Raw news content or search results
    :return: A formatted summary of the news
    """
    # Parse the news content from search results
    try:
        # If news_content is JSON string, parse it
        if news_content.startswith('[') or news_content.startswith('{'):
            data = json.loads(news_content)
        else:
            # If it's plain text, treat as single article
            data = [{"title": "News Article", "content": news_content, "url": ""}]
        
        # Ensure data is a list
        if isinstance(data, dict):
            data = [data]
        
        summaries = []
        for i, article in enumerate(data[:3]):  # Limit to top 3 articles
            title = article.get('title', f'Article {i+1}')
            content = article.get('content', article.get('snippet', ''))
            url = article.get('url', '')
            
            # Extract main points (first 200 chars of content)
            main_points = content[:200] + "..." if len(content) > 200 else content
            
            summary = f"""
    📰 **{title}**
    🔗 {url}
    📝 **Summary:** {main_points}
    """
            summaries.append(summary)
        
        final_summary = "📊 **NEWS SUMMARY**\n" + "="*50 + "\n"
        final_summary += "\n".join(summaries)
        
        return final_summary
        
    except Exception as e:
        return f"Error processing news content: {str(e)}"
    pass

# TODO: Add to tools list and test with:
# "Find recent news about artificial intelligence and give me a summary"

In [7]:
# Initialize the Tavily search tool
search = TavilySearchResults()

@tool
def search_tool(query: str):
    """
    Search the web for information using Tavily API.

    :param query: The search query string
    :return: Search results related to the query
    """
    return search.invoke(query)

In [8]:
tools=[search_tool, news_summarizer_tool]
tools_by_name={ tool.name:tool for tool in tools}

In [9]:
# Configure ChatOpenAI for Azure OpenAI
deployment_name = "gpt-4o-mini"  # Replace with your actual deployment name

openai_llm = AzureChatOpenAI(
    azure_deployment=deployment_name,
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    temperature=0.7,
    model_name=deployment_name,
    max_retries=3,
    request_timeout=120,
    max_tokens=1024,
    top_p=0.95   
)


In [12]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """
You are a helpful AI assistant that thinks step-by-step and uses tools when needed.

When responding to queries:
1. First, think about what information you need
2. Use available tools if you need current data or specific capabilities  
3. Provide clear, helpful responses based on your reasoning and any tool results

Always explain your thinking process to help users understand your approach.
"""),
    MessagesPlaceholder(variable_name="scratch_pad")
])

# binding the tools to the LLM
model_react = chat_prompt | openai_llm.bind_tools(tools)

# Agent state:


class AgentState(TypedDict):
    """The state of the agent."""
    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]



In [13]:
user_query = "Find recent AI news and summarize the top 3 articles"

In [14]:
# tool execution node
def tool_node(state: AgentState):
    """Execute all tool calls from the last message in the state."""
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# model invocation node
def call_model(state: AgentState):
    """Invoke the model with the current conversation state."""
    response = model_react.invoke({"scratch_pad": state["messages"]})
    return {"messages": [response]}


# decision logic node
def should_continue(state: AgentState):
    """Determine whether to continue with tool use or end the conversation."""
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

then the graph itself:


In [15]:
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Add edges between nodes
workflow.add_edge("tools", "agent")  # After tools, always go back to agent

# Add conditional logic
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",  # If tools needed, go to tools node
        "end": END,          # If done, end the conversation
    },
)

# Set entry point
workflow.set_entry_point("agent")

# Compile the graph
graph = workflow.compile()

run/test it

In [16]:
def print_stream(stream):
    """Helper function for formatting the stream nicely."""
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [HumanMessage(content=user_query)]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Find recent AI news and summarize the top 3 articles
================================== Ai Message ==================================
Tool Calls:
  search_tool (call_Y87XMu0XU9ac8dbiV1FOupji)
 Call ID: call_Y87XMu0XU9ac8dbiV1FOupji
  Args:
    query: recent AI news
================================= Tool Message =================================
Name: search_tool

[{"title": "AI News September 2025: In-Depth and Concise - The AI Track", "url": "https://theaitrack.com/ai-news-september-2025-in-depth-and-concise/", "content": "## Latest AI News [...] The AI Track\nThe AI Track\nThe AI Track\nThe AI Track\n\n# AI News September 2025: In-Depth and Concise\n\n## Welcome to The AI Track's comprehensive monthly roundup of the latest AI news!\n\nEach month, we compile significant news, trends, and happenings in AI, providing detailed summaries with key points in bullet form for concise yet complete understanding. 